In [44]:
import json
import uuid
import os
import json
from dotenv import load_dotenv
from pathlib import Path
from kafka import KafkaProducer
from faker import Faker
from time import sleep

In [45]:
dotenv_path = Path('/resources/.env')
load_dotenv(dotenv_path=dotenv_path)

True

In [46]:
kafka_host = os.getenv('KAFKA_HOST')
kafka_topic = os.getenv('KAFKA_TOPIC_NAME')
# kafka_topic_partition = os.getenv('KAFKA_TOPIC_NAME')+"-1"
kafka_topic_partition = "customer_transactions"

In [47]:
from kafka import KafkaConsumer

# Kafka Consumer Setup
consumer = KafkaConsumer(
    group_id='dibimbing-group',
    bootstrap_servers=[f"{kafka_host}:9092"],
    value_deserializer=lambda m: json.loads(m.decode("utf-8")),
    auto_offset_reset="earliest",  # Start from the beginning if no offset is stored
    enable_auto_commit=False,  # Manual commit
    max_poll_records=10,  # Reduce batch size (default: 500)
    session_timeout_ms=60000,  # Increase timeout for stability
    max_poll_interval_ms=300000  # Increase time before rebalancing
)


In [48]:
from kafka import TopicPartition
consumer.assign([
    TopicPartition(kafka_topic_partition, 0),
    TopicPartition(kafka_topic_partition, 1)
])

In [49]:
try:
    while True:
        messages = consumer.poll(timeout_ms=1000)  # Poll frequently
        if not messages:
            continue  # No new messages, keep polling

        for tp, records in messages.items():
            for record in records:
                data = record.value
                print(f"Received Transaction: {data}")

        consumer.commit()  # Commit offsets only after processing
except KeyboardInterrupt:
    print("Consumer stopped.")
finally:
    consumer.close()

Received Transaction: {'transaction_id': 'e1e2b169-7822-4dd4-a243-6f7f86b687b2', 'customer_name': 'Brittany Norris', 'category': 'Electronics', 'amount': 4913.96, 'payment_type': 'Crypto', 'location': 'Austinborough', 'timestamp': 879385544}
Received Transaction: {'transaction_id': '38da8888-bae9-4c9c-ab57-98c3f62da8bc', 'customer_name': 'Kristen Taylor', 'category': 'Fashion', 'amount': 6910.61, 'payment_type': 'Cash', 'location': 'Alvarezbury', 'timestamp': 818273904}
Received Transaction: {'transaction_id': '0c1d68f8-fb92-4c47-93f8-dbf0c2ecbab0', 'customer_name': 'Shawn Jordan', 'category': 'Groceries', 'amount': 13462.07, 'payment_type': 'Cash', 'location': 'East Jacquelineborough', 'timestamp': 303731388}
Received Transaction: {'transaction_id': 'f36f2efc-7628-4369-8336-88f4626c131b', 'customer_name': 'Lauren Walker', 'category': 'Groceries', 'amount': 11381.6, 'payment_type': 'Credit Card', 'location': 'Harristown', 'timestamp': 1155405790}
Received Transaction: {'transaction_id'